In [ ]:
import sqlite3
import pandas as pd
import datetime

# Connect to the database
conn = sqlite3.connect('Investment Returns.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define the starting and ending points for the loop
starting_quarter = 1
starting_year = 18
ending_quarter = 4
ending_year = 23

# Define the ER Number With Type to search
er_number_with_type = '000000K'

# Define the list of table prefixes
table_prefixes = ['BYP', 'NWB', 'REN']

# Define the list to store the query results
results = []

# Loop through the specified tables and execute the queries
for prefix in table_prefixes:
    for year in range(starting_year, ending_year + 1):
        for quarter in range(starting_quarter, ending_quarter + 1):
            # Define the table name
            table_name = f"{prefix}{quarter}Q{year}"
            
            # Check if the table exists before executing the query
            cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
            if cursor.fetchone() is None:
#                 print(f"Table {table_name} does not exist. Skipping...")
                continue
            
            # Define the SQL query
            sql_query = f'''
                SELECT *, '{table_name}' AS "Table Name"
                FROM {table_name}
                WHERE "ER Number With Type" = '{er_number_with_type}'
            '''
            
            # Execute the query and fetch the results
            cursor.execute(sql_query)
            table_results = cursor.fetchall()
            
            # If rows are found, add them to the results list
            if table_results:
                results.extend(table_results)

# Close the cursor and database connection
cursor.close()
conn.close()

# Define column names
column_names = ["Name", "ER Number With Type", "SumOfGA INV EARNINGS", "SumOfGA CLOS BAL EE", 
                "SumOfGA CLOS BAL ER", "SumOfSA INV EARNINGS", "SumOfSA CLOS BAL EE",
                "SumOfSA CLOS BAL ER", "FileName"]

# Create a pandas DataFrame from the query results with column headers
df = pd.DataFrame(results, columns=column_names)

# Print the DataFrame
display(df)

In [ ]:
df_returns = df.copy()
df_returns = df_returns[["Name", "ER Number With Type", "FileName"]]
df_returns

In [ ]:
# Calculate the opening balance using shifted values

df_returns["Opening Balance"] = (
    df["SumOfGA CLOS BAL EE"].shift()
    + df["SumOfGA CLOS BAL ER"].shift()
    + df["SumOfSA CLOS BAL EE"].shift()
    + df["SumOfSA CLOS BAL ER"].shift()
)

df_returns["Closing Balance"] = (
    df["SumOfGA CLOS BAL EE"]
    + df["SumOfGA CLOS BAL ER"]
    + df["SumOfSA CLOS BAL EE"]
    + df["SumOfSA CLOS BAL ER"]
)



# Apply the function to calculate Investment Earnings
df_returns["Investment Earnings"] = df_returns.apply(calculate_investment_earnings, axis=1)

# Calculate Returns
df_returns["Returns"] = (
    (df_returns["Investment Earnings"] * 2)
    / (df_returns["Opening Balance"] + df_returns["Closing Balance"] - df_returns["Investment Earnings"])
)


# Apply the formatting function to the "Returns" column
df_returns["Returns"] = df_returns["Returns"].apply(format_percent)

# Display the modified 'df_returns'
display(df_returns)



In [ ]:
# Define a function to convert quarter and year to starting and ending dates
def convert_to_dates(file_name):
    quarter = int(file_name[-4])  # Extract the quarter (last character)
    year = int(file_name[-2:]) + 2000  # Extract the year (last two characters) and add 2000
    
    # Calculate starting and ending dates based on quarter and year
    starting_date = f"{year}-{(quarter - 1) * 3 + 1:02d}-01"
    ending_date = f"{year}-{quarter * 3:02d}-" + str(pd.Timestamp(year, quarter * 3, 1).daysinmonth)
    
    return pd.Series([starting_date, ending_date])

# Apply the function to "FileName" column and create new columns
dates_df = df_returns["FileName"].apply(convert_to_dates)
df_returns[["Starting Dates", "Ending Dates"]] = dates_df

# Define the generate_dates function to calculate days between start and end dates
def generate_dates(start_date, end_date):
    diff = (end_date - start_date).days + 1
    return diff

# Convert columns to datetime
df_returns["Starting Dates"] = pd.to_datetime(df_returns["Starting Dates"])
df_returns["Ending Dates"] = pd.to_datetime(df_returns["Ending Dates"])

# Apply the generate_dates function to calculate days and populate the "Days" column
df_returns["Days"] = df_returns.apply(lambda row: generate_dates(row["Starting Dates"], row["Ending Dates"]), axis=1)

# Print the modified DataFrame
display(df_returns)

In [ ]:
# print(f"For plan {df_returns['Name'][0]}, the estimated returns are:")
df_returns = df_returns[["Starting Dates", "Ending Dates", "Days", "Opening Balance", "Returns"]]
slicing_done = False
if not slicing_done:
    df_returns = df_returns[1:]
    slicing_done = True
display(df_returns)
for i, value in enumerate(df_returns["Returns"]):
    if i < len(df_returns["Returns"]) - 1:
        print(f"'{value}',", end=" ")
    else:
        print(f"'{value}'", end="")
        
# Define a boolean flag to track whether the slicing has been performed
slicing_done = False

# Check if the slicing has not been done yet
# if not slicing_done:
#     # Perform the slicing operation only once
#     df_returns = df_returns[1:]
#     slicing_done = True
